In [1]:
from fastai import *
from fastai.text import *
from fastai.docs import *
import pandas as pd
from tqdm import tqdm
import pickle

from sklearn.model_selection import train_test_split
YELP_PATH = "../../data/yelp/"
YELP_PATH

'../../data/yelp/'

In [2]:
#Running this in python 3.7.0 will result in segment fault due to a bug in the underlying C code related to the data structures
def train_test_separator_gen(generator): #Next best thing
    progress = 0
    for chunk in generator:
        chunk = chunk.drop(columns=['business_id','cool','date','funny','review_id', 'useful', 'user_id'])
        chunk.columns = ['0','1']
        test = chunk.sample(frac=0.2)
        with open(r"../../data/yelp/train.csv", 'a', encoding="utf-8") as trainFile:
            chunk.to_csv(trainFile, header=(progress==0), encoding="utf-8", index=False)
        with open(r"../../data/yelp/test.csv", 'a', encoding="utf-8") as testFile:
            test.to_csv(testFile, header=(progress==0), encoding="utf-8", index=False)
        progress+=1

dataset = pd.read_json(YELP_PATH+'review.json',chunksize=100, lines=True, encoding='utf-8')
train_test_separator_gen(dataset)

ValueError: Expected object or value

In [2]:
train = pd.read_csv(YELP_PATH+"test.csv", chunksize=5)
next(train).head()
#print("Train")
#dataset.to_csv(YELP_PATH+"test.csv")

,0,1
0,3,I've been to Ellis Island a handful of times I...
1,5,Truly the only only HK Cafe worth going in the...
2,5,Sakura was fabulous! By Outback Steakhouse Har...
3,5,This was an awesome experience. We did have t...
4,5,Great haircut. Great stylist. Ask for Karen.Te...


In [2]:
classes = read_classes(YELP_PATH+'classes.txt')
classes

['0star', '1star', '2star', '3star', '4star', '5star']

In [ ]:
data_lm = text_data_from_csv(Path(YELP_PATH), data_func=lm_data, chunksize=40000, bs=32)
print('done')

Tokenizing train.


Numericalizing train.


In [5]:
pickle.dump(data_lm, open("data_lm","wb"))

In [ ]:
data_clas = text_data_from_csv(Path(YELP_PATH), data_func=classifier_data, vocab=data_lm.train_ds.vocab, chunksize=40000, bs=32)
print('done')

In [ ]:
pickle.dump(data_clas, open("data_clas","wb"))

In [15]:
download_wt103_model()

In [ ]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

In [7]:
learn.save_encoder('ft_enc')

In [8]:
pred = learn.get_preds(is_test=True)
pred

TypeError: object of type 'NoneType' has no len()

In [5]:
learn = RNNLearner.classifier(data_clas)
learn.load_encoder('ft_enc')
learn.fit(1, 1e-3)

KeyboardInterrupt: 

In [ ]:
learn.save_encoder('ft_enc')